## ORES comparison for clustered vital 1k + links (`Edcast 65`) dataset

**Author:** Jim Maddock

**Last Updated:** 6-9-20

**Description:** The purpose of this notebook is compare the categories (clusters) from the EdCast 65 taxonomy to the categories predicted by ORES.  This notebook will:

* Categorize all articles in each cluster, then look at distribution of categories in each cluster - **DONE**
* Categorize all articles in each cluster and assign a label based on the most frequent category. Look at whether the edcast label makes sense compared to the ORES label.
* Categorize all articles.  Look at number of clusters that appear in each ORES predicted category

We can run these evaluations using 2 different ORES datasets:

* **ORES predicted topics:** Overlap with this dataset shows us how the edcast model compares to ORES *predictions*.  The advantage is that we can use a much larger dataset (the entirety of wikipedia).  The disadvantage is that these are predictions, so we can't necessarily assume that ORES represents the "correct" or ground truth answer. - **DONE**
* **ORES training data:** Overlap with this dataset shows how the edcast model compares to human a human crafted taxonomy.  This dataset is much smaller, but we can assume that all articles are correctly categorized. - **DONE**

In [1]:
import json
import pandas as pd
from oresapi import Session
import requests

import matplotlib.pyplot as plt
import matplotlib

matplotlib.style.use('ggplot')
pd.set_option("display.min_rows", 100)
pd.set_option("display.max_rows", 100)

### Notebook Setup
Import edcast data from json file and create pandas dataframe.  Import ORES predictions from https://figshare.com/articles/Topics_for_each_Wikipedia_Article_across_Languages/12127434 merge with edcast dataframe.

In [2]:
FILEPATH = '/Users/klogg/research_data/wmf_knowledge_graph/wiki_5-28-20/wiki_65_clusters_6-8-20.json'

with open(FILEPATH) as json_file:
    cluster_65 = json.load(json_file)

In [4]:
df = pd.DataFrame()

for i, cluster in enumerate(cluster_65):
    chunk = []
    for article in cluster_65[cluster]['items']:
        row = {
            'label':cluster_65[cluster]['label'],
            'cluster':cluster_65[cluster]['cluster'],
            'w':article['w'],
            'title':article['title']
        }
        chunk.append(row)
    df = df.append(pd.DataFrame(chunk))
    #print('finished cluster: {0}'.format(i))
    
df['title'] = df['title'].apply(lambda x: x.replace(' ','_'))

In [5]:
FILEPATH = '/Users/klogg/research_data/wmf_knowledge_graph/topics/topicsForAllWikipediaPages.csv'

topics = pd.read_csv(FILEPATH,escapechar='\\')

In [6]:
topics = topics.loc[topics['wiki_db'] == 'enwiki']
topics = topics.rename(columns={'page_title':'title'})
topics = topics.sort_values(['title','probability'],ascending=False)
topics = topics.reset_index(drop=True).drop_duplicates(subset = ['title','Qid'],keep='first')

In [111]:
df = df.merge(topics, on='title', how='left')

### EdCast + ORES topic prediction comparison
**Number of ORES topics per EdCast cluster:** All EdCast clusters are categorized as containing a large number of the availible topics, between 46 and 87 percent.  This indcates that the edcast model is **not** providing the same categorization as ORES.

In [196]:
topic_dist = df.groupby('label')['topic'].nunique().describe().to_frame('count')
topic_dist['percent'] = topic_dist['count'].divide(df['topic'].nunique())*100
topic_dist

,count,percent
count,65.000000,103.174603
mean,41.861538,66.446886
std,5.719694,9.078880
min,29.000000,46.031746
25%,38.000000,60.317460
50%,42.000000,66.666667
75%,46.000000,73.015873
max,55.000000,87.301587


**Distribution of number of topics per edcast cluster:** When we look at the distribution of the number of articles in each ORES predicted topic:

**positive:**
* most of these clusters have a long-tail distribution, which means that even though all clusters contain between 46 and 87 percent of topics, most of these topics less than 1 percent of the total articles in the cluster.
* a few edcast clusters belong predominantly to a single topic
    * e.g. "numb_function_theory_space" is 74% "STEM.STEM*"
    * "plant_forest_species_tree" is 72% "STEM.STEM*"
    
**negative:**
* the median "max percentage" is ~34%, which means that most edcast clusters are at least split into thirds
* the min is 10%, which means "desert_oblast_comedy_earthquake" is split among more than 10 large categories

In [199]:
percent_df = df.groupby(['label','topic']).size().to_frame('count').reset_index()
percent_df['sum'] = percent_df.groupby('label')['count'].transform(sum)
percent_df['percent'] = (percent_df['count'].divide(percent_df['sum']))*100
percent_df.sort_values(['label','percent'])
percent_df.groupby('label')['percent'].describe().sort_values('max')

,count,mean,std,min,25%,50%,75%,max
label,,,,,,,,
desert_oblast_comedy_earthquake,47.0,2.127660,2.495351,0.359712,0.719424,1.079137,2.338129,10.431655
list_country_poet_egypt,47.0,2.127660,2.519334,0.247525,0.495050,1.237624,2.722772,11.138614
river_mountain_peninsula_north,44.0,2.272727,3.138498,0.094787,0.189573,0.805687,2.606635,11.469194
company_new_medium_oil,51.0,1.960784,3.122045,0.112613,0.450450,0.675676,2.083333,15.878378
museum_national_art_history,41.0,2.439024,3.554079,0.303951,0.303951,0.911854,2.735562,17.629179
stone_temple_architecture_circle,40.0,2.500000,3.532064,0.124533,0.373599,0.809465,3.237858,17.808219
greek_ancient_roman_list,36.0,2.777778,4.865407,0.149254,0.410448,0.671642,2.089552,18.656716
day_year_calendar_child,46.0,2.173913,3.941620,0.163399,0.326797,0.653595,2.287582,19.444444
city_province_list_prefecture,46.0,2.173913,3.742886,0.070077,0.280308,0.700771,2.645410,19.691661


**Percentage of articles per Edcast cluster that belong to the largest predicted ORES category:** I.E the median value is 34% which indicates that in 50% of EdCast clusters, the largest predicted ORES category contains less 34% of the total articles in that cluster. 

In [200]:
percent_df.groupby('label')['percent'].describe()['max'].describe()

count    65.000000
mean     37.101882
std      16.157437
min      10.431655
25%      24.260355
50%      34.195065
75%      46.204620
max      74.202899
Name: max, dtype: float64

**Percentage of articles per Edcast cluster that belong to the median predicted ORES category:** I.E the median value is .59% which indicates that in 50% of EdCast clusters, the middle sized predicted ORES category contains less .59% of the total articles in that cluster. 

In [204]:
percent_df.groupby('label')['percent'].describe()['50%'].describe()

count    65.000000
mean      0.595174
std       0.246013
min       0.211416
25%       0.409417
50%       0.587084
75%       0.711382
max       1.237624
Name: 50%, dtype: float64

### EdCast + ORES topic prediction samples

**Number of predicted ORES categories for each EdCast cluster**:

In [120]:
df.groupby('label')['topic'].nunique().sort_values()

label
cycle_bicycle_wheel_drive                     29
cell_protein_system_receptor                  30
syndrome_heart_artery_disease                 31
physic_quantum_theory_particle                31
star_variable_stellar_supernova               32
dialect_german_language_min                   33
acid_carbon_chemistry_reaction                34
numb_function_theory_space                    34
greek_ancient_roman_list                      36
space_list_solar_moon                         37
equation_garden_law_force                     37
time_comet_clock_great                        38
religion_god_mythology_jewish                 38
genetic_evolution_tourism_population          38
drug_management_theatre_policy                38
disorder_emotional_syndrome_sleep             38
island_sea_gulf_list                          38
al_ibn_islam_islamic                          38
chinese_buddhism_china_buddhist               38
de_list_jean_la                               39
script_braille

**Articles and predicted topics in the "cycle_bicycle_wheel_drive" EdCast cluster:**

In [133]:
df.loc[df['label'] == 'cycle_bicycle_wheel_drive'][['title','topic']]

,title,topic
25775,Cycling,Culture.Sports
25776,Outline_of_bicycles,NaN
25777,Recumbent_bicycle,Culture.Sports
25778,Touring_bicycle,Culture.Sports
25779,Motorized_bicycle,Culture.Sports
25780,Bicycle_touring,Culture.Sports
25781,Penny-farthing,STEM.STEM*
25782,Freight_bicycle,Culture.Sports
25783,Utility_bicycle,Culture.Sports
25784,Safety_bicycle,Culture.Sports


**Number of Articles in each ORES predicted topic in the "cycle_bicycle_wheel_drive" EdCast cluster:**
* This is super long-tailed.  All but 3 of the 29 categories have less than 5 articles
* Most of the articles belong to STEM or Sports, which is because articles are either about bicycles or biological/chemical cycles (see the cell below this one for a more detailed view)

In [205]:
df.loc[df['label'] == 'cycle_bicycle_wheel_drive'].groupby('topic').size().sort_values()

topic
STEM.Earth_and_environment                      1
History_and_Society.Politics_and_government     1
Culture.Performing_arts                         1
History_and_Society.History                     1
Geography.Regions.Oceania                       1
Culture.Visual_arts.Architecture                1
Culture.Visual_arts.Fashion                     1
Geography.Regions.Europe.Western_Europe         1
Geography.Regions.Africa.Africa*                1
Geography.Regions.Americas.South_America        1
Geography.Regions.Asia.East_Asia                1
Geography.Regions.Europe.Southern_Europe        1
Culture.Biography.Biography*                    2
History_and_Society.Society                     2
Geography.Regions.Europe.Northern_Europe        2
Geography.Regions.Europe.Europe*                2
Culture.Media.Media*                            2
STEM.Space                                      2
STEM.Technology                                 2
History_and_Society.Business_and_economics  

In [139]:
df.loc[(df['label'] == 'cycle_bicycle_wheel_drive') & (df['topic'] == 'STEM.STEM*')]

,label,cluster,w,title,Qid,topic,probability,page_id,wiki_db
25781,cycle_bicycle_wheel_drive,28,0.606544,Penny-farthing,Q1554036,STEM.STEM*,0.79,183351.0,enwiki
25786,cycle_bicycle_wheel_drive,28,0.584621,Atkinson_cycle,Q384952,STEM.STEM*,0.99,519209.0,enwiki
25791,cycle_bicycle_wheel_drive,28,0.541644,Ericsson_cycle,Q384539,STEM.STEM*,0.98,826277.0,enwiki
25792,cycle_bicycle_wheel_drive,28,0.540536,Miller_cycle,Q657886,STEM.STEM*,1.00,74761.0,enwiki
25801,cycle_bicycle_wheel_drive,28,0.495984,Steering_wheel,Q679300,STEM.STEM*,0.92,772013.0,enwiki
25806,cycle_bicycle_wheel_drive,28,0.488096,Wheel,Q446,STEM.STEM*,0.99,33555.0,enwiki
25815,cycle_bicycle_wheel_drive,28,0.466507,Kleemenko_cycle,Q6420078,STEM.STEM*,1.00,19871306.0,enwiki
25816,cycle_bicycle_wheel_drive,28,0.466175,Stirling_cycle,Q910550,STEM.STEM*,1.00,247323.0,enwiki
25818,cycle_bicycle_wheel_drive,28,0.463178,Spoke,Q845671,STEM.STEM*,0.71,170836.0,enwiki
25820,cycle_bicycle_wheel_drive,28,0.454256,Glyoxylate_cycle,Q575119,STEM.STEM*,1.00,3322454.0,enwiki


**Number of Articles in each ORES predicted topic in the "state_unite_union_liberalism" EdCast cluster:**  It's possible to see why most of these articles would be related, but we're probably not producing better categories.  Might be worth validating this by hand.

In [206]:
df.loc[df['label'] == 'state_unite_union_liberalism'].groupby('topic').size().sort_values()

topic
STEM.Space                                       1
STEM.Mathematics                                 1
STEM.Physics                                     1
STEM.Libraries_&_Information                     1
Geography.Regions.Africa.Western_Africa          1
Culture.Media.Entertainment                      1
Geography.Regions.Africa.Southern_Africa         1
STEM.Computing                                   1
Culture.Internet_culture                         1
Geography.Regions.Africa.Eastern_Africa          1
Culture.Performing_arts                          2
Culture.Sports                                   2
Culture.Visual_arts.Architecture                 2
History_and_Society.Education                    2
Culture.Visual_arts.Visual_arts*                 2
Culture.Food_and_drink                           2
STEM.Engineering                                 2
History_and_Society.Transportation               3
STEM.Technology                                  3
Culture.Media.Music      

In [147]:
df.loc[(df['label'] == 'state_unite_union_liberalism') & (df['topic'] == 'History_and_Society.Society')]

,label,cluster,w,title,Qid,topic,probability,page_id,wiki_db
46041,state_unite_union_liberalism,49,0.645006,Self-determination,Q156595,History_and_Society.Society,1.00,29269.0,enwiki
46042,state_unite_union_liberalism,49,0.641665,Voting_rights_in_the_United_States,Q405566,History_and_Society.Society,1.00,667785.0,enwiki
46070,state_unite_union_liberalism,49,0.600630,Freedom_of_religion_in_the_United_States,Q2142732,History_and_Society.Society,1.00,7493150.0,enwiki
46091,state_unite_union_liberalism,49,0.581370,Mass_surveillance,Q1425056,History_and_Society.Society,1.00,331195.0,enwiki
46097,state_unite_union_liberalism,49,0.578095,Arab_Spring,Q33761,History_and_Society.Society,0.52,30655949.0,enwiki
46122,state_unite_union_liberalism,49,0.565052,Freedom_of_movement,Q1344824,History_and_Society.Society,1.00,1270497.0,enwiki
46124,state_unite_union_liberalism,49,0.563690,McCarthyism,Q207066,History_and_Society.Society,0.99,43805.0,enwiki
46155,state_unite_union_liberalism,49,0.548138,Commonwealth_of_Nations,Q7785,History_and_Society.Society,1.00,21175158.0,enwiki
46162,state_unite_union_liberalism,49,0.546457,United_Nations_Human_Rights_Council,Q205650,History_and_Society.Society,0.99,635790.0,enwiki
46169,state_unite_union_liberalism,49,0.542284,Forced_disappearance,Q1288449,History_and_Society.Society,1.00,686148.0,enwiki


**Percentages (rather than absolute values) for each ORES category in a few EdCast clusters:** This reflects the earlier distribution analysis in this notebook.

In [187]:
percent_df.loc[percent_df['label'] == 'al_ibn_islam_islamic'].sort_values('percent')

,label,topic,count,sum,percent
58,al_ibn_islam_islamic,Geography.Regions.Europe.Northern_Europe,1,614,0.162866
36,al_ibn_islam_islamic,Culture.Food_and_drink,1,614,0.162866
37,al_ibn_islam_islamic,Culture.Internet_culture,1,614,0.162866
69,al_ibn_islam_islamic,STEM.Physics,1,614,0.162866
50,al_ibn_islam_islamic,Geography.Regions.Africa.Eastern_Africa,1,614,0.162866
67,al_ibn_islam_islamic,STEM.Libraries_&_Information,1,614,0.162866
42,al_ibn_islam_islamic,Culture.Media.Music,1,614,0.162866
43,al_ibn_islam_islamic,Culture.Media.Television,1,614,0.162866
47,al_ibn_islam_islamic,Culture.Visual_arts.Visual_arts*,1,614,0.162866
45,al_ibn_islam_islamic,Culture.Sports,1,614,0.162866


In [188]:
percent_df.loc[percent_df['label'] == 'book_comic_fiction_publish'].sort_values('percent')

,label,topic,count,sum,percent
160,book_comic_fiction_publish,STEM.Technology,1,840,0.119048
143,book_comic_fiction_publish,Geography.Regions.Europe.Eastern_Europe,1,840,0.119048
138,book_comic_fiction_publish,Geography.Regions.Asia.Central_Asia,1,840,0.119048
144,book_comic_fiction_publish,Geography.Regions.Europe.Europe*,1,840,0.119048
134,book_comic_fiction_publish,Geography.Regions.Americas.Central_America,1,840,0.119048
133,book_comic_fiction_publish,Geography.Regions.Africa.Eastern_Africa,1,840,0.119048
148,book_comic_fiction_publish,Geography.Regions.Oceania,1,840,0.119048
141,book_comic_fiction_publish,Geography.Regions.Asia.Southeast_Asia,1,840,0.119048
151,book_comic_fiction_publish,History_and_Society.Politics_and_government,1,840,0.119048
154,book_comic_fiction_publish,STEM.Computing,1,840,0.119048


In [202]:
percent_df.loc[percent_df['label'] == 'plant_forest_species_tree'].sort_values('percent')

,label,topic,count,sum,percent
1779,plant_forest_species_tree,STEM.Space,1,1419,0.070472
1777,plant_forest_species_tree,STEM.Physics,1,1419,0.070472
1743,plant_forest_species_tree,Culture.Internet_culture,1,1419,0.070472
1775,plant_forest_species_tree,STEM.Libraries_&_Information,1,1419,0.070472
1745,plant_forest_species_tree,Culture.Literature,1,1419,0.070472
1773,plant_forest_species_tree,STEM.Chemistry,1,1419,0.070472
1747,plant_forest_species_tree,Culture.Media.Media*,1,1419,0.070472
1748,plant_forest_species_tree,Culture.Media.Music,1,1419,0.070472
1771,plant_forest_species_tree,History_and_Society.Transportation,1,1419,0.070472
1766,plant_forest_species_tree,History_and_Society.Business_and_economics,1,1419,0.070472


In [5]:
FILEPATH = '/Users/klogg/research_data/wmf_knowledge_graph/topics/labeled_enwiki_with_topics_metadata.json'

with open(FILEPATH) as json_file:
    ground_truth_topics = [json.loads(line) for line in json_file]

In [18]:
dt_list = []

for article in ground_truth_topics:
    if len(article['topics']) > 0:
        article_dict = {
            'topic':article['topics'][0],
            'title':article['title']
        }
        dt_list.append(article_dict)

dt_df = pd.DataFrame(dt_list)

In [23]:
dt_df = dt_df.merge(df, on='title', how='inner')

In [31]:
topic_dist = dt_df.groupby('label')['topic'].nunique().describe().to_frame('count')
topic_dist['percent'] = topic_dist['count'].divide(dt_df['topic'].nunique())*100
topic_dist

,count,percent
count,65.000000,162.500000
mean,33.323077,83.307692
std,3.553641,8.884103
min,23.000000,57.500000
25%,31.000000,77.500000
50%,33.000000,82.500000
75%,37.000000,92.500000
max,40.000000,100.000000


In [35]:
percent_df = dt_df.groupby(['label','topic']).size().to_frame('count').reset_index()
percent_df['sum'] = percent_df.groupby('label')['count'].transform(sum)
percent_df['percent'] = (percent_df['count'].divide(percent_df['sum']))*100
percent_df.sort_values(['label','percent'])
percent_df.groupby('label')['percent'].describe().sort_values('max')

,count,mean,std,min,25%,50%,75%,max
label,,,,,,,,
computer_image_photography_weapon,40.0,2.500000,2.105449,0.046992,0.787124,2.020677,3.724154,8.599624
water_lake_environmental_soil,37.0,2.702703,3.469289,0.100301,0.401204,1.203611,3.209629,12.236710
energy_power_system_nuclear,37.0,2.702703,3.094059,0.099502,0.597015,1.492537,3.383085,13.233831
list_country_poet_egypt,38.0,2.631579,2.871447,0.125628,0.785176,1.507538,3.831658,13.819095
state_unite_union_liberalism,38.0,2.631579,4.261322,0.046404,0.197216,0.556845,2.053364,14.013921
company_new_medium_oil,40.0,2.500000,3.737443,0.098232,0.368369,0.884086,2.111984,15.324165
stone_temple_architecture_circle,33.0,3.030303,4.559115,0.113122,0.339367,0.791855,2.941176,16.742081
war_military_force_world,36.0,2.777778,4.730019,0.099701,0.299103,0.747757,2.467597,19.641077
dive_air_engine_gas,36.0,2.777778,4.066206,0.107527,0.403226,1.397849,3.252688,20.860215


In [36]:
percent_df.groupby('label')['percent'].describe()['max'].describe()

count    65.000000
mean     39.785857
std      17.003745
min       8.599624
25%      27.654321
50%      39.667897
75%      49.113592
max      81.135707
Name: max, dtype: float64

In [37]:
percent_df.groupby('label')['percent'].describe()['50%'].describe()

count    65.000000
mean      0.802699
std       0.403644
min       0.229885
25%       0.534759
50%       0.737101
75%       1.060071
max       2.020677
Name: 50%, dtype: float64

In [39]:
dt_df.groupby('label')['topic'].nunique().sort_values()

label
church_jesus_pope_john                        23
star_variable_stellar_supernova               27
script_braille_alphabet_write                 27
dialect_german_language_min                   27
al_ibn_islam_islamic                          28
cell_protein_system_receptor                  28
cuisine_food_list_milk                        29
plant_forest_species_tree                     29
french_france_battle_war                      30
russia_russian_sea_soviet                     30
religion_god_mythology_jewish                 30
cycle_bicycle_wheel_drive                     30
genetic_evolution_tourism_population          31
language_english_linguistic_consonant         31
numb_function_theory_space                    31
island_sea_gulf_list                          31
physic_quantum_theory_particle                31
film_animation_cinema_list                    31
poetry_literature_animal_poet                 31
book_comic_fiction_publish                    31
desert_oblast_

In [40]:
len(dt_list)

5511929